# DEPENDÊNCIAS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import warnings


from google.colab import drive
from itertools import product
from sklearn.base import BaseEstimator, clone
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# FUNÇÕES

## Constantes

In [3]:
_PARAMS = "params"
_CROSS_VAL = "cv"
_TEST = "test"

## Utilitário

In [4]:
def dict_flatten(dictionary: dict, fields_to_flatten: list[str]) -> dict:
    """
    Desestrutura os campos selecionados de um dicionário, retornando apenas os atributos desejados.

    :param dictionary: Dicionário original que contém os campos a serem desestruturados.
    :param fields_to_flatten: Lista de nomes dos campos que devem ser desestruturados (flatten).

    :return: Novo dicionário contendo apenas os atributos dos campos selecionados, combinados em um nível.
    """
    dict_flat = dict()

    for field in fields_to_flatten:
        value = dictionary.get(field)
        if isinstance(value, dict):
            dict_flat.update(value)

    return dict_flat

def dict_to_flat_df(dictionary: dict, fields_to_flatten: list[str], key_name: str = "key") -> pd.DataFrame:
    """
    Converte um dicionário com estrutura aninhada em um DataFrame tabular, desestruturando
    apenas os campos especificados.

    :param dictionary: Dicionário em que cada chave representa um identificador único e cada valor é um dicionário com possíveis campos aninhados.
    :param fields_to_flatten: Lista de nomes de campos a serem desestruturados. Cada um deve
                               corresponder a uma chave cujo valor é um dicionário.
    :param key_name: Nome da coluna que armazenará os identificadores do dicionário original.
                     Padrão é "key".

    :return: DataFrame com uma coluna para os identificadores e colunas adicionais contendo os
             atributos resultantes da desestruturação dos campos selecionados.
    """
    rows = list()

    for identifier, nested_dict in dictionary.items():
        dict_flat = dict_flatten(nested_dict, fields_to_flatten)
        dict_flat[key_name] = identifier
        rows.append(dict_flat)

    df = pd.DataFrame(rows)
    df = df[[key_name] + [col for col in df.columns if col != key_name]]  # Coloca a chave primeiro
    return df

In [5]:
def dict_deserialize(path: str) -> dict:
    """
    Desserializa (carrega) um dicionário Python a partir de um arquivo .pkl.

    :param path: Caminho completo do arquivo a ser carregado.
                 Deve terminar com a extensão '.pkl' (ex: 'dict_cases.pkl').
    :return: Dicionário Python carregado a partir do arquivo.
    """
    return joblib.load(path)

def dict_serialize(dictionary: dict, path: str) -> None:
    """
    Serializa (salva) um dicionário Python em um arquivo no formato .pkl.

    :param dictionary: Dicionário a ser serializado.
    :param path: Caminho completo do arquivo de destino.
                 Deve terminar com a extensão '.pkl' (ex: 'dict_cases.pkl').
    """
    joblib.dump(dictionary, path)

def generate_sheet(dict_cases: dict, sheet_name: str, path: str) -> None:
    """
    Gera uma planilha Excel (.xlsx) contendo os dados tabulares dos casos modelados.

    :param dict_cases: Dicionário onde cada chave identifica um caso e o valor é um
                       dicionário com os dados do modelo, parâmetros e métricas.
    :param sheet_name: Nome da aba (sheet) que será criada no arquivo Excel.
    :param path: Caminho completo do arquivo .xlsx que será criado.
                 Deve terminar com a extensão '.xlsx' (ex: 'resultados.xlsx').

    A função realiza o seguinte:
    - Achata os campos definidos em FLATTEN (campos dicionário), elevando-os ao primeiro nível.
    - Concatena os dados em um único DataFrame.
    - Escreve esse DataFrame em uma planilha Excel na aba especificada.
    """
    FLATTEN = [_PARAMS, _CROSS_VAL, _TEST] # Campos a serem passados para o primeiro nível
    KEY_NAME = "case" # Nome do campo que armazenará o identificador dos casos

    """
    Achata campos que são dicionários, deixando todas as informações no primeiro
    nível. Em seguida, transforma em um DataFrame.
    """
    df_final = dict_to_flat_df(dict_cases, FLATTEN, KEY_NAME)

    with pd.ExcelWriter(path, engine="openpyxl") as writer:
        df_final.to_excel(writer, sheet_name=sheet_name, index=False)

# SETUP DO AMBIENTE

In [6]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# SETUP

In [7]:
# Constantes simbólicas
PRE = "pre_processing"              # Pré-processamento
RF = "random_forest"                # Random Forest
XGB = "xgboost"                     # XGBoost
MLR = "multiple_linear_regression"  # Regressão Linear Múltipla
ALL = "all"                         # Todos os modelos considerados simultaneamente

# Dicionário para armazenar os resultados
dict_cases = dict() # Armazena dicionários de casos
dict_df_cases = dict() # Armazena DataFrames de casos

# SALVAMENTO
PATH_IN = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/src_sprint03/plans"
PATH_OUT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/src_sprint03/plans"

In [8]:
for key in [PRE, MLR, XGB]:
    dict_cases[key] = dict_deserialize(f"{PATH_IN}/{key}.pkl")
    dict_df_cases[key] = pd.DataFrame(dict_cases[key]).T
    generate_sheet(dict_cases[key], key, f"{PATH_OUT}/{key}.xlsx")
    print(f"Planilha {key} gerada.")

Planilha pre_processing gerada.
Planilha multiple_linear_regression gerada.


EOFError: 

# TESTES LEITURA DE PKL

In [ ]:
PATH_BASE = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/s
# for i, k in zip(range(1), dict_pre.keys()):
#     X = dict_pre[k]["X"]
#     y = dict_pre[k]["y"]
#     params = dict_pre[k]["params"]

# display(X.head())
# display(y.head())
# display(y.describe())
# display(params)

# # STATUS: OK!
# dict_mlr = dict_deserialize(f"{PATH_BASE}/mlinreg.pkl")

# for i, k in zip(range(1), dict_mlr.keys()):
#     dict_models = dict_mlr[k]["best_model"]
#     dict_scores_cross_val = dict_mlr[k]["cv"]
#     dict_scores_test = dict_mlr[k]["test"]

# print(dict_models)

# plot_coefs(dict_models["r2_cv_best_model"].coef_)

# Teste Resíduos

In [ ]:
PATH_BASE = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/src_sprint03/plans"
dict_mlr = dict_deserialize(f"{PATH_BASE}/multiple_linear_regression.pkl")
dict_pre = dict_deserialize(f"{PATH_BASE}/pre_processing.pkl")

for i, k in zip(range(1), dict_mlr.keys()):
    dict_models = dict_mlr[k]["best_model"]
    dict_scores_cross_val = dict_mlr[k]["cv"]
    dict_scores_test = dict_mlr[k]["test"]
    r2_y_pred_test = dict_mlr[k]["y_pred"]["r2_best_model_test_y_pred"]
    # print(dict_mlr[k])
    model = dict_models["r2_cv_best_model"]
    print(model)
    model_name = "_" + model.__class__.__name__ + "_"
    print(model_name)
    print(k)

    k_pre = k.split(model_name)[0]
    print(k_pre)
    k_pre = k_pre
    print(k_pre)

    print(k_pre)

    X = dict_pre[k_pre]['X']
    y = dict_pre[k_pre]['y']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
    # y_pred = model.predict(X_test)
    y_pred = r2_y_pred_test

    residues = y_test - y_pred
    print(r2_score(y_test, y_pred))
    display(residues)

Lasso(alpha=0.0001, random_state=42)
_Lasso_
FS_na_FE_na_OUT0_NORM0_PCA5_Lasso_ALPHA_0.0001_FIT_INTERCEPT_true_MAX_ITER_1000_RANDOM_STATE_42
FS_na_FE_na_OUT0_NORM0_PCA5
FS_na_FE_na_OUT0_NORM0_PCA5
FS_na_FE_na_OUT0_NORM0_PCA5
0.397371083721644


,que_area_queimada
527,1.082322e+06
359,5.006538e+04
447,-6.290061e+04
31,5.048390e+04
621,-3.176296e+05
...,...
668,-1.906047e+05
239,2.370618e+04
312,9.607568e+04
211,-3.662560e+04
